In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install required libraries
!pip install -U sentence-transformers

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 3.0 MB/s eta 0:0

In [33]:
import torch
from torch.utils.data import DataLoader, Dataset
from sentence_transformers import SentenceTransformer, losses
import pandas as pd
import matplotlib.pyplot as plt

# Define paths to datasets in Google Drive
train_path = '/content/drive/My Drive/train.csv'
dev_path = '/content/drive/My Drive/dev.csv'
# Function to check if a string starts with an alphabet
import pandas as pd
import torch
from torch.utils.data import Dataset


# Read CSV files and filter out rows that start with an alphabet
df = pd.read_csv(train_path, sep='\t', on_bad_lines='skip')
df['score'] = df['score'].astype(str)
df = df[~df['score'].str.startswith(tuple(chr(i) for i in range(65, 91)))]

df2 = pd.read_csv(dev_path, sep='\t', on_bad_lines='skip')
df2['score'] = df2['score'].astype(str)
df2 = df2[~df2['score'].str.startswith(tuple(chr(i) for i in range(65, 91)))]

# Convert DataFrames to list of dictionaries
data = df.dropna().to_dict('records')
data2 = df2.dropna().to_dict('records')

# Define custom dataset class
class Dataset1c(Dataset):
    def __init__(self, data):
        self.s1 = [d['sentence1'] for d in data]
        self.s2 = [d['sentence2'] for d in data]
        self.score = [d['score'] for d in data]

    def __len__(self):
        return len(self.score)

    def __getitem__(self, index):
        cos_similarity = torch.tensor(float(self.score[index]), dtype=torch.float32)
        return InputExample(texts=[self.s1[index], self.s2[index]], label=cos_similarity)

# Usage
dataset_train = Dataset1c(data)
dataset_dev = Dataset1c(data2)
model = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [34]:
import torch
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
import pandas as pd
import matplotlib.pyplot as plt


dataloader_1c = DataLoader(dataset_train , batch_size=16, shuffle=True)


# Define loss function
train_loss = losses.CosineSimilarityLoss(model=model)

model.fit(
    [(dataloader_1c, train_loss)],
    epochs=2,
)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/357 [00:00<?, ?it/s]

Iteration:   0%|          | 0/357 [00:00<?, ?it/s]

In [35]:
# Save the trained model checkpoint to a file
checkpoint_path = '/content/drive/My Drive/checkpoint2.pth'
model.save(checkpoint_path)

# Download the checkpoint file
from google.colab import files
files.download(checkpoint_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
from sentence_transformers import SentenceTransformer, evaluation
from sentence_transformers import SentenceTransformer, util

# # Load the trained model
# model_path = '/content/drive/My Drive/checkpoint.pth'
# trained_model = SentenceTransformer(model_path)

# Load dev data
dev_data = pd.read_csv(dev_path, sep='\t')
sentences1_dev = dev_data['sentence1'].tolist()
sentences2_dev = dev_data['sentence2'].tolist()
scores_dev = dev_data['score'].tolist()

# Compute embedding for both lists
embeddings1 = model.encode(sentences1_dev, convert_to_tensor=True)
embeddings2 = model.encode(sentences2_dev, convert_to_tensor=True)

# Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)



In [37]:
# Scale cosine similarity scores from 0-1 to 0-5
scaled_scores = [score * 5 for score in cosine_scores]


In [39]:
pred_scalar = [torch.mean(tensor).item() for tensor in scaled_scores]
print(pred_scalar)

[3.782538414001465, 3.6714389324188232, 4.168054580688477, 3.556894540786743, 3.5597639083862305, 3.799091339111328, 4.065691947937012, 3.9247756004333496, 3.8565008640289307, 3.868492603302002, 3.9353694915771484, 3.7596209049224854, 4.128787517547607, 3.9885470867156982, 3.706141471862793, 3.792520761489868, 3.680609703063965, 3.7116730213165283, 3.9198970794677734, 3.631006956100464, 3.8632819652557373, 3.7596209049224854, 3.7047746181488037, 3.7434585094451904, 3.8663182258605957, 3.6662979125976562, 4.057916641235352, 4.124919891357422, 3.6116995811462402, 3.993849754333496, 3.843362808227539, 3.476209878921509, 4.1618170738220215, 3.7596209049224854, 3.761993885040283, 3.78277325630188, 3.8017282485961914, 3.9013562202453613, 3.632662296295166, 3.8974905014038086, 3.7479360103607178, 3.9384677410125732, 4.08991813659668, 3.3477752208709717, 3.7833364009857178, 3.8851735591888428, 3.6523780822753906, 3.746241331100464, 3.9530513286590576, 4.08436918258667, 3.6578259468078613, 3.82

In [38]:
import torch
from scipy.stats import pearsonr

def calculate_pearson_correlation(predictions, ground_truth):

    pred_scalar = [torch.mean(tensor).item() for tensor in predictions]
    pearson_corr, _ = pearsonr(pred_scalar, ground_truth)
    return pearson_corr

# Usage example
pred = scaled_scores
ground_truth = scores_dev

pearson_corr = calculate_pearson_correlation(pred, ground_truth)
print("Pearson Correlation Coefficient:", pearson_corr)


Pearson Correlation Coefficient: 0.06045105903612337
